# Ансамбли моделей машинного обучения.

## Задание 

1. Выберите набор данных (датасет) для решения задачи классификации или регресии.
2. В случае необходимости проведите удаление или заполнение пропусков и кодирование <br>категориальных признаков.
3. С использованием метода train_test_split разделите выборку на обучающую и <br>тестовую.
4. Обучите две ансамблевые модели. Оцените качество моделей с помощью одной из <br>подходящих для задачи метрик. Сравните качество полученных моделей.
5. Произведите для каждой модели подбор значений одного гиперпараметра. <br>В зависимости от используемой библиотеки можно применять функцию GridSearchCV, <br>использовать перебор параметров в цикле, или использовать другие методы.
6. Повторите пункт 4 для найденных оптимальных значений гиперпараметров. <br>Сравните качество полученных моделей с качеством моделей, полученных в пункте 4.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
data=pd.read_csv("Admission_Predict.csv",sep=",")
data.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [3]:
y = data['Research']
x = data.drop('Research', axis = 1)

### Разделим выборку на тренировочную и тестовую

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42)

### Обучим модели и  проверим данные используя <br> метрики для классификации

#### Случайный лес

In [5]:
random_forest = RandomForestClassifier(
    n_estimators=10, max_depth=1, 
    random_state=0).fit(X_train, y_train)

In [6]:
res_RF = random_forest.predict(X_test)
print(accuracy_score(y_test, res_RF))
print(precision_score(y_test, res_RF))

0.7375
0.90625


#### Градиентный бустинг

In [7]:
gradient_boosting = GradientBoostingClassifier(
    n_estimators=10, max_depth=10, 
    learning_rate=0.01).fit(X_train, y_train)

In [8]:
res_GB = gradient_boosting.predict(X_test)
print(accuracy_score(y_test, res_GB))
print(precision_score(y_test, res_GB))

0.7125
0.7222222222222222


### Подберем гиперпараметры, <br>обучим модели и  проверим данные используя <br> метрики для классификации

#### Случайный лес

In [9]:
parameters_random_forest = {'n_estimators':[1, 3, 5, 7, 10], 
                            'max_depth':[1, 3, 5, 7, 10],
                            'random_state':[0, 2, 4, 6, 8, 10]}
best_random_forest = GridSearchCV(RandomForestClassifier(), 
                                  parameters_random_forest, cv=3, 
                                  scoring='accuracy')
best_random_forest.fit(X_train, y_train)

/Users/lina/Documents/ML-Technology/.venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [1, 3, 5, 7, 10], 'max_depth': [1, 3, 5, 7, 10], 'random_state': [0, 2, 4, 6, 8, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [10]:
best_random_forest.best_params_

{'max_depth': 3, 'n_estimators': 10, 'random_state': 10}

In [11]:
new_RF = RandomForestClassifier(n_estimators=5, 
                                max_depth=3, 
                                random_state=10).fit(X_train, y_train)

In [12]:
new_res_RF = new_RF.predict(X_test)
print(accuracy_score(y_test, new_res_RF))
print(precision_score(y_test, new_res_RF))

0.775
0.8372093023255814


#### Градиентный бустинг

In [13]:
parameters_gradient_boosting = {'n_estimators':[1, 3, 5, 7, 10], 
                            'max_depth':[1, 3, 5, 7, 10],
                            'learning_rate':[0.001, 0.0025, 0.005, 0.0075, 0.01, 0.025]}
best_gradient_boosting = GridSearchCV(GradientBoostingClassifier(), parameters_gradient_boosting, 
                                      cv=3, scoring='accuracy')
best_gradient_boosting.fit(X_train, y_train)

/Users/lina/Documents/ML-Technology/.venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_sampl...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [1, 3, 5, 7, 10], 'max_depth': [1, 3, 5, 7, 10], 'learning_rate': [0.001, 0.0025, 0.005, 0.0075, 0.01, 0.025]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [14]:
best_gradient_boosting.best_params_

{'learning_rate': 0.025, 'max_depth': 1, 'n_estimators': 10}

In [15]:
new_GB = GradientBoostingClassifier(n_estimators=10, 
                                    max_depth=3, 
                                    learning_rate=0.025).fit(X_train, y_train)

In [16]:
new_res_GB = new_GB.predict(X_test)
print(accuracy_score(y_test, new_res_GB))
print(precision_score(y_test, new_res_GB))

0.75
0.8461538461538461
